# Topics


In [3]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time

In [4]:
sys.setrecursionlimit(1500)

In [5]:
with open("04-input", "r") as file:
    lines = file.readlines()
data_raw = [line.replace("\n", "") for line in lines]
data_raw = "\n".join(data_raw)
data_raw

'XMASAXMASXXMMAMAXAXXXMSASXSXSXMAMMAMXMASASXMXMASMSMSSMMSXMASXMSMXMASMMSSMMMSMMAMMSMMMMMSXMAXMSMMMMMSAXAXXXMASAMXXSXMXSSSMSMSXMAMXXXSAXSMSAMX\nMSMMMMSMXMXSSSSSSMSMSMSASAAXXASAXXMAXSMSAMMSAMSXAAAAXSAMXMASXMXMASAMXXAAAXMAMMAAMAMXAASAMMMSAMXMAASMSXMSMSMXMASMXMASXMAAXAAAXXSSSMMAMMXAXSXX\nAAAAAAAMMSAAMAAAXAAAAAMMMMMXMXMASXMSASXMAMAMASAMSMMMXMXSAMXSXMASXSAMXMSSMMSASXXSSMMSAMMASAAAAXXXSMSAAMXAAMSXSAMASAAMAMAMSMSMSMAAASAAXAMXMMMS\nSSSMMSSSSMSSMMMMMSMXMMMAAXXAXAMXMXAMASMSMMMSMMMXASAMXSMSXSAMAXAMAMASAMXAXXMAMXAAMXXMXMMAMMSSSMXXXAMXMAMMSMAXMASXXMASMMAXAAAXAAMMMMSXXSAMXAAA\nXMAXMXXMAAAXXXAMXXSMMMSMMSAMXMMMMMXMXMAAMAASAMXSAXXMAXAMXMMSSMSSSSMMASXMMMMAMSSMSMMAAXMASXAAAAASMMMSXSXAXMAMSAMXSSMSXMAMMXMSMSSXSAMXAAXXSMSS\nXMASMMASMMMSSSMMSAMXSAAAAAXMAMASAMASXMSMSMXSASXXSMSMSSSSXXAAAAAAXAASAMASXXMMXAAXXAMMASAMXMMMMSMXAAXMAMMMMMXMMASAAAXMASXMSMAMXAAXMASAMXMMAAAM\nXAXAAAAXAXAAAAMSMAMAMSSMMSMSMSASASASAXAAXMASAMMAMSXMAAAXMMMSMMMMSSMMXSXMMAMSMMSMSXMMAAAMMXSAAAMSMMSMAMAMAMASXMMMSMMMAMAAAAAMMMSMMAMMSAMXMMMS\nMSXSM

In [72]:
test_data_raw = r"""MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""


test_data_raw2 = r"""..X...
.SAMX.
.A..A.
XMAS.S
.X...."""

def preprocess_data (data):
    # dtype='U10'
    data_trans = [list(row) for row in data.split("\n")]
    data_trans = np.asarray(data_trans)
    # for row in data.split("\n"):   
    #     reports.append(np.array([int(item) for item in row.split(" ")]))

    return data_trans
test_data = preprocess_data(test_data_raw)
test_data2 = preprocess_data(test_data_raw2)
display(test_data)

array([['M', 'M', 'M', 'S', 'X', 'X', 'M', 'A', 'S', 'M'],
       ['M', 'S', 'A', 'M', 'X', 'M', 'S', 'M', 'S', 'A'],
       ['A', 'M', 'X', 'S', 'X', 'M', 'A', 'A', 'M', 'M'],
       ['M', 'S', 'A', 'M', 'A', 'S', 'M', 'S', 'M', 'X'],
       ['X', 'M', 'A', 'S', 'A', 'M', 'X', 'A', 'M', 'M'],
       ['X', 'X', 'A', 'M', 'M', 'X', 'X', 'A', 'M', 'A'],
       ['S', 'M', 'S', 'M', 'S', 'A', 'S', 'X', 'S', 'S'],
       ['S', 'A', 'X', 'A', 'M', 'A', 'S', 'A', 'A', 'A'],
       ['M', 'A', 'M', 'M', 'M', 'X', 'M', 'M', 'M', 'M'],
       ['M', 'X', 'M', 'X', 'A', 'X', 'M', 'A', 'S', 'X']], dtype='<U1')

In [12]:
data = preprocess_data(data_raw)
data

array([['X', 'M', 'A', ..., 'A', 'M', 'X'],
       ['M', 'S', 'M', ..., 'S', 'X', 'X'],
       ['A', 'A', 'A', ..., 'M', 'M', 'S'],
       ...,
       ['M', 'X', 'X', ..., 'M', 'S', 'M'],
       ['S', 'A', 'A', ..., 'S', 'A', 'M'],
       ['M', 'X', 'S', ..., 'M', 'A', 'S']], dtype='<U1')

In [86]:
def solution (data, verbose=False):

    def search_line (row):
        pattern = r"XMAS"
        matches_for = re.findall(pattern, row)
        matches_back = re.findall(pattern, row[::-1])
        
        
        return len(matches_for) + len(matches_back)

    total = 0
    
    for x in range(data.shape[0]):
        row = "".join(data[x,:].tolist())
        found = search_line(row)
        total += found
    for y in range(data.shape[1]):
        row = "".join(data[:,y].tolist())
        found = search_line(row)
        total += found
    for z in range(-data.shape[1]+1, data.shape[0]-1):
        row = "".join(data.diagonal(z).tolist())
        found = search_line(row)
        total += found
        
        row = "".join(np.fliplr(data).diagonal(z).tolist())
        found = search_line(row)
        total += found
        
    return total




sol = solution(test_data, verbose=True)

print(sol)


18


In [87]:
sol = solution(data)
print(sol)

2560


# Part 2

In [91]:
def solution2 (data, verbose=False):


    def array_str(array):
        return "".join(array.tolist())

    def calc_options(x, y):
        options = []
        
        
        if x >=1  and x < data.shape[0]-1 and y >= 1 and y < data.shape[1]-1:
            area = data[x-1:x+2,y-1:y+2]
            if (array_str(area.diagonal()) == "MAS" or array_str(area.diagonal()) == "SAM") \
                and (array_str(np.fliplr(area).diagonal()) == "MAS" or array_str(np.fliplr(area).diagonal()) == "SAM"):
                return 1
        
        return 0
                
        
    total = 0
    for x in range(data.shape[0]):   
        for y in range(data.shape[1]):   
            total += calc_options(x,y)

    return total




sol = solution2(test_data, verbose=True)
print(sol)
# print(dists)

# display(sum(sol))

    

9


In [92]:
sol = solution2(data)
print(sol)

1910
